In [1]:
import numpy as np
import pandas as pd
import os.path as path
import random

data_path = "../data/"
results_path = "../results/archive/"
kernels = pd.read_csv(data_path+'kernel_list.csv')
runtimes_path = results_path+"T4_18-6-3_sm52.csv"
run_times = pd.read_csv(runtimes_path,low_memory=False)
# Pathnames cleanup
run_times['path'] = run_times['path'].apply(lambda x: x.replace('step2/', data_path+'kernels/').replace('/main.cu', ''))

Add the file path to the dataframe

In [2]:
kernels['kernel_path'] = [data_path+"kernels/"+str(r["Repo"])+"/"+str(r["underdirectory"]) 
                        for _, r in kernels.iterrows()]

Explore the dataset

In [3]:
kernels.head()

,Repo,underdirectory,function,status,variables,kernel_path
0,1,0,euclidean_kernel,runs,"[('const float *', 'vg_a'), ('size_t', 'pitch_...",../data/kernels/1/0
1,1,1,euclidean_kernel_same,runs,"[('const float *', 'vg_a'), ('size_t', 'pitch_...",../data/kernels/1/1
2,1,2,maximum_kernel,runs,"[('const float *', 'vg_a'), ('size_t', 'pitch_...",../data/kernels/1/2
3,1,3,maximum_kernel_same,runs,"[('const float *', 'vg_a'), ('size_t', 'pitch_...",../data/kernels/1/3
4,1,4,manhattan_kernel,runs,"[('const float *', 'vg_a'), ('size_t', 'pitch_...",../data/kernels/1/4


In [4]:
print(len(kernels))

20258


In [5]:
run_times.head()

,path,function,blocks,matrix,time
0,../data/kernels/1/0,euclidean_kernel,"(1, 1024)","(1016, 1016)",2970.211154
1,../data/kernels/1/0,euclidean_kernel,"(1, 1024)","(1232, 1232)",2786.452857
2,../data/kernels/1/0,euclidean_kernel,"(1, 1024)","(16, 16)",3124.981053
3,../data/kernels/1/0,euclidean_kernel,"(1, 1024)","(1680, 1680)",2591.644286
4,../data/kernels/1/0,euclidean_kernel,"(1, 1024)","(2024, 2024)",2476.894286


Try to create a ProGraML graph from LLVM IR compiled kernels

In [6]:
CREATE_GRAPHS = False

In [7]:
import programl
import glob
import os
from tqdm import tqdm

Using backend: pytorch


In [8]:
files = glob.glob(os.path.join(data_path, 'kernels', '*', '*', '*.ll'), recursive=True)
if CREATE_GRAPHS:
    num_created_graphs = 0
    print(f"Found {len(files)} LLVM-IR files to convert")
    for i in tqdm(range(len(files))):
        llvm_filename = files[i]
        function_name = llvm_filename.split('/')[-1].split('.')[0]
        kernel_path = os.path.dirname(llvm_filename)
        graph_filename = os.path.join(kernel_path, f'{function_name}.graph')
        if not os.path.exists(graph_filename):
            # Open the LLVM-IR file
            with open(files[i], 'r') as f:
                llvm_program = f.read()
            # Create the ProgramGraph object from the LLVM-IR. 
            # Use a timeout of 5 seconds to filter out large programs
            # and avoid locks.
            try:
                graph = programl.from_llvm_ir(llvm_program, timeout=5)
            # Write a representation of the graphs to a file
            except TimeoutError:
                print(f'Timeout while processing graph for {files[i]}')
                continue
            except programl.GraphCreationError as e:
                print(f'Error while processing graph for {files[i]}: {e}')
                continue
            # Filter out graphs with only 1 node
            if len(graph.node) > 1:
                # If no error occured, write graph to file
                programl.save_graphs(graph_filename, [graph])
                # The graph was created successfully
                num_created_graphs += 1
else:
    graph_files = glob.glob(os.path.join(data_path, 'kernels', '*', '*', '*.graph'), recursive=True)
    print(f"Found {len(files)} LLVM-IR files")
    print(f"Found {len(graph_files)} graph files")

Found 20044 LLVM-IR files
Found 20043 graph files


Create full list and small list of valid indexes to use in the kernels dataset and for each create their training and testval sets

In [9]:
full_idxs = [   
    i
    for i, gpath in enumerate(
    (kernels['kernel_path'] + os.sep + kernels['function'] + '.graph').unique())
    if os.path.exists(gpath)
]

full_train = random.sample(full_idxs, int(len(full_idxs)*0.8))
sfullt = set(full_train)
full_test = [i for i in full_idxs if i not in sfullt]

len(full_idxs), len(full_train), len(full_test)

(20043, 16034, 4009)

In [10]:
small_idxs = random.sample(full_idxs, 1000)

small_train = random.sample(small_idxs, int(len(small_idxs)*0.8))
ssmallt = set(small_train)
small_test = [i for i in small_idxs if i not in ssmallt]

len(small_idxs), len(small_train), len(small_test)

(1000, 800, 200)

In [11]:
import pickle
import json

with open('../data/full_idx_list.pickle', 'wb') as f:
    pickle.dump(full_idxs, f)
with open('../data/lscat_idxs.json', 'w') as f:
    json.dump({'train': full_train, 'testval': full_test}, f)

with open('../data/small_idx_list.pickle', 'wb') as f:
    pickle.dump(small_idxs, f)
with open('../data/small_lscat_idxs.json', 'w') as f:
    json.dump({'train': small_train, 'testval': small_test}, f)

In [12]:
kernels = kernels.iloc[full_idxs]

The matrix sizes and block sizes that are reported are:

In [13]:
allowed_blocks=['(8, 8)','(16, 16)'  ,'(24, 24)','(32, 32)','(1, 64)','(1, 128)',
                '(1, 192)','(1, 256)','(1, 320)','(1, 384)','(1, 448)','(1, 512)',
                '(1, 576)','(1, 640)','(1, 704)','(1, 768)','(1, 832)','(1, 896)',
                '(1, 960)','(1, 1024)']
allowed_matrix=['(240, 240)','(496, 496)','(784, 784)','(1016, 1016)',
                '(1232, 1232)','(1680, 1680)','(2024, 2024)']

In [14]:
ok_matrix = run_times[run_times['matrix'].isin(allowed_matrix)].index
ok_block  = run_times[run_times['blocks'].isin(allowed_blocks)].index
ok_index  = ok_matrix.intersection(ok_block)
proc_run_times = run_times.iloc[ok_index]

In [15]:
print(f'Removed {len(run_times) - len(proc_run_times)} unrelated rows. Left {len(proc_run_times)} rows')

Removed 402141 unrelated rows. Left 2531763 rows


In [16]:
prior_len = len(proc_run_times)
proc_run_times = proc_run_times.loc[np.logical_and(proc_run_times['path'].isin(kernels['kernel_path']),
                                    proc_run_times['function'].isin(kernels['function']))]
print(f'Removed {prior_len - len(proc_run_times)} unrelated rows. Left {len(proc_run_times)} rows')

Removed 27605 unrelated rows. Left 2504158 rows


In [213]:
df = proc_run_times.set_index(['path', 'function', 'matrix', 'blocks']).unstack('blocks')
df = df.reindex(columns=sorted(df.columns, key=lambda x: x[::-1]))
df.columns = ['block_{}'.format(t) for _, t in df.columns]
df.fillna(np.inf, inplace=True)

In [222]:
def get_labels(idx: int, const: float) -> np.ndarray:
    row = proc_run_times.iloc[idx]
    times = df.loc[row['path'], row['function'], row['matrix']]
    # Transf. #1
    s = times.min() / times
    # Transf. #2
    s = np.exp(s * const)
    s /= np.sum(s)
    s /= s.max()
    return pd.DataFrame(data=[s, times], index=['scores', 'times'])

get_labels(random.randint(0, len(proc_run_times)), 12)

,"block_(1, 1024)","block_(1, 128)","block_(1, 192)","block_(1, 256)","block_(1, 320)","block_(1, 384)","block_(1, 448)","block_(1, 512)","block_(1, 576)","block_(1, 64)","block_(1, 640)","block_(1, 704)","block_(1, 768)","block_(1, 832)","block_(1, 896)","block_(1, 960)","block_(16, 16)","block_(24, 24)","block_(32, 32)","block_(8, 8)"
scores,0.889278,0.200706,0.203355,0.0587,0.155228,0.217967,0.107404,0.019002,0.218687,0.227385,0.784427,0.903877,0.247189,0.987621,1.00,0.924597,0.279163,0.375211,0.071239,0.199088
times,2552.813333,2918.410000,2914.733333,3309.9000,2992.383333,2895.426667,3105.200000,3774.406667,2894.516667,2883.783333,2580.053333,2549.320000,2861.070000,2530.476667,2527.85,2544.473333,2828.616667,2752.716667,3241.426667,2920.683333
